In [79]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [80]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [81]:
spark

In [82]:
data_folder = '/labs/laba01/ml-100k'
estimations_file_name = 'u.data'
films_file_name = 'u.item'
path_to_estimation_file = os.path.join(data_folder, estimations_file_name)
path_to_films_file = os.path.join(data_folder, films_file_name)

In [83]:
! hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [84]:
estimations = spark.sparkContext.textFile(path_to_estimation_file)
films = spark.sparkContext.textFile(path_to_films_file)

In [85]:
from operator import methodcaller
from typing import List

def split_raw_data(sentences: List[str], delimeter: str):
    return list(map(methodcaller("split", delimeter), sentences))

In [86]:
# Структура датасета: user id | item id | rating | timestamp
print("Вид образца датасета: \n", estimations.first(), "\n")
print("Обработанный датасет: \n", split_raw_data(estimations.take(10), "\t"), "\n")

Вид образца датасета: 
 196	242	3	881250949 

Обработанный датасет: 
 [['196', '242', '3', '881250949'], ['186', '302', '3', '891717742'], ['22', '377', '1', '878887116'], ['244', '51', '2', '880606923'], ['166', '346', '1', '886397596'], ['298', '474', '4', '884182806'], ['115', '265', '2', '881171488'], ['253', '465', '5', '891628467'], ['305', '451', '3', '886324817'], ['6', '86', '3', '883603013']] 



In [87]:
# Структура датасета: movie id | movie title | release date | video release date |
#               IMDb URL | unknown | Action | Adventure | Animation |
#               Children's | Comedy | Crime | Documentary | Drama | Fantasy |
#               Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
#               Thriller | War | Western |
print("Вид образца датасета: \n", films.first(), "\n")
print("Обработанный датасет: \n", split_raw_data(films.take(10), "|"), "\n")

Вид образца датасета: 
 1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0 

Обработанный датасет: 
 [['1', 'Toy Story (1995)', '01-Jan-1995', '', 'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['2', 'GoldenEye (1995)', '01-Jan-1995', '', 'http://us.imdb.com/M/title-exact?GoldenEye%20(1995)', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0'], ['3', 'Four Rooms (1995)', '01-Jan-1995', '', 'http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0'], ['4', 'Get Shorty (1995)', '01-Jan-1995', '', 'http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['5', 'Copycat (1995)', '01-Jan

In [88]:
# Персонализация
test_film_id = '328'

In [94]:
hist_film = estimations.map(
    methodcaller("split", "\t")
).map(
    lambda n: ((n[1], n[2]), 1)  # [(('film_id', 'estim'), 1),]
).reduceByKey(
    lambda estim1, estim2: estim1 + estim2  # [(('film_id', 'estim'), count),]
).map(
    lambda n: (n[0][0], (n[0][1], n[1]))  # [(film_id', ('estim', count)),]
).filter(
    lambda n: n[0] == test_film_id  # [('film_id', ('estim', count)),] where film_id == test_film_id
).map(
    lambda n: n[1]  # [('estim', count),]
).sortBy(
    lambda n: n[0]  # by 'estim' asc
).map(
    lambda n: n[1]  # only count
).collect()

In [123]:
from operator import itemgetter

hist_all = list(map(
    itemgetter(1),
    sorted(
        estimations.map(
            methodcaller("split", "\t")
    ).map(
        lambda n: (n[2], 1)  # [(estim', 1),]
    ).countByKey().items(), key=lambda n: n[0], reverse=False)))

# Или так
hist_all = estimations.map(
    methodcaller("split", "\t")
).map(
    lambda n: (n[2], 1)  # [(('film_id', 'estim'), 1),]
).reduceByKey(
    lambda estim1, estim2: estim1 + estim2  # [(estim', count),]
).sortBy(
    lambda n: n[0]  # by 'estim' asc
).map(
    lambda n: n[1]  # only count
).collect()

In [127]:
result = {"hist_film": hist_film, "hist_all": hist_all}

In [128]:
import json
with open("lab01.json", "w") as outfile:
    json.dump(result, outfile)

In [ ]:
spark.stop()